In [4]:
import os
import sys
from helpers.logger import LoggerHelper, logging
from helpers.config import ConfigHelper
from helpers import config
from helpers import logger
import numpy as np
import pandas as pd
from classes import preprocessing_functions
from classes.multiclass_models import NeuralNetwork, ConvolutionalNeuralNetwork, RecurrentNeuralNetwork, LongShortTermMemory
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from modules.lib import *
from modules.optimizer import *
import json


In [9]:
with open('workflows_all.json') as f:
    json_data = json.load(f)


In [ ]:
pd.read_csv('metadata/proxy_data_models/')

In [3]:
indicator_list = ["f3"]

X, Y, Z = preprocessing_functions.read_data('data_subset', indicator_list)
#2 not anomalous
#0 electrical
#1 mechanical


2024-10-23 10:32:41,467 - classes.preprocessing_functions - INFO - read_data(): Read data from path data_subset
2024-10-23 10:32:41,470 - classes.preprocessing_functions - INFO - read_data(): 1 variables are read: ['f3']
2024-10-23 10:33:06,945 - classes.preprocessing_functions - INFO - read_data(): Number of files read 648


In [4]:
X_pad = preprocessing_functions.add_padding(X, indicator_list)

Y_encoded = preprocessing_functions.encode_response_variable(Y)



2024-10-23 10:33:06,982 - classes.preprocessing_functions - INFO - add_padding(): Matching the length of the time series adding padding
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [5]:
X_train, X_test, y_train, y_test,z_train,z_test = preprocessing_functions.split_data(X_pad, Y_encoded, Z)

n_timestamps = X_train.shape[1]
n_features = X_train.shape[2]

n_classes = y_train.shape[1]


In [6]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from keras.layers import Masking, Dense, Flatten, Conv1D, MaxPooling1D, SimpleRNN, LSTM
from keras import Input

def create_model(n_timestamps, n_features, activation_function='relu', units=(50, 50,50), n_classes=2):
    model = Sequential()
    # Input layer
    model.add(Input(shape=(n_timestamps, n_features)))
    # Masking layer
    model.add(Masking(mask_value=0))
    # Fully connected layers
    for unit in units:
        model.add(Dense(units=unit, activation=activation_function))
    # Flatten layer
    model.add(Flatten())
    # Output layer
    model.add(Dense(n_classes, activation="softmax"))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# model = KerasClassifier(build_fn=create_model, 
#                         n_timestamps=n_timestamps, 
#                         n_features=n_features, 
#                         n_classes=n_classes, 
#                         verbose=0)

# Define the parameter grid
param_grid = {
    'model__units': [[100, 100, 100],[256, 256, 128],[512,512,512]],     # Example values
    'model__activation_function': ['relu','tanh'],       # Example values
    'batch_size': (16,32,64) ,                        # Example values
    'epochs': (10,50),                       # Example values
}

# # Initialize GridSearchCV
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2)
# grid_result = grid.fit(X_train, y_train)

In [14]:
# trained_models = {}
# from itertools import product

# combinations = list(product(param_grid['model__units'], 
#                             param_grid['model__activation_function'], 
#                             param_grid['batch_size'], 
#                             param_grid['epochs']))

# # Loop through each combination of hyperparameters
# for i, (units, activation_function, batch_size, epochs) in enumerate(combinations, 1):
#     print(f"Workflow {i} with units={units}, activation={activation_function}, batch_size={batch_size}, epochs={epochs}")
    
# #     # Create and train the model
# #     model = create_model(n_timestamps=n_timestamps, n_features=n_features, activation_function=activation_function, units=units, n_classes=3)
    
# #     model.fit(X_train, y_train, 
# #               batch_size=batch_size, 
# #               epochs=epochs)
    
# #     # Store the trained model in the dictionary with key i (model number)
# #     trained_models[i] = model

# # # Save the trained models dictionary to a .pkl file
# # with open('trained_models.pkl', 'wb') as file:
# #     pickle.dump(trained_models, file)

In [9]:
# import dill as pickle
# with open('metadata/proxy_data_models/Ideko_nn_grid_new.pkl', 'wb') as file:
#     pickle.dump(grid_result, file)

In [17]:
import dill as pickle

with open('metadata/proxy_data_models/Ideko_nn_grid_new.pkl', 'rb') as f:
                            original_model = pickle.load(f)

In [13]:
import grpc
import xai_service_pb2
import xai_service_pb2_grpc
with grpc.insecure_channel('localhost:50051') as channel:
    stub = xai_service_pb2_grpc.ExplanationsStub(channel)
    k = stub.GetExplanation(xai_service_pb2.ExplanationsRequest(explanation_type='hyperparameterExplanation',explanation_method='2dpdp',workflows=str(json_data), model='Ideko_model',feature1='epochs',feature2='units'))


In [11]:
import grpc
import xai_service_pb2
import xai_service_pb2_grpc
with grpc.insecure_channel('localhost:50051') as channel:
    stub = xai_service_pb2_grpc.ExplanationsStub(channel)
    k = stub.GetExplanation(xai_service_pb2.ExplanationsRequest(explanation_type='hyperparameterExplanation',explanation_method='counterfactuals',model='Ideko_model'))


In [12]:
preds = np.argmax(original_model.best_estimator_.predict(X_test), axis = 1)

In [13]:
labels = np.argmax(y_test, axis = 1)

In [14]:
predictions = pd.DataFrame(columns=['Predictions','Labels'])

In [26]:
predictions['Predictions'] = list(preds)
predictions['Labels'] = list(labels)

In [27]:
proxy = pd.DataFrame(columns = ['hyperparameters','Label'])
for i,params_dict in enumerate(original_model.cv_results_['params']):
    # mdl = deepcopy(original_model.estimator)
    # mdl.set_params(**params_dict)
    # mdl.fit(X_train, y_train)
    # prediction = np.argmax(mdl.predict([X_test[3]]),axis=1)
    proxy = proxy.append({'hyperparameters' : params_dict},ignore_index=True)


In [28]:
proxy['Label'] = 1

# Set the first 6 values to 2
proxy.loc[:8, 'Label'] = 2

# Set the next 4 values to 1
proxy.loc[8:14, 'Label'] = 0

In [30]:
keys = list(proxy['hyperparameters'].iloc[0].keys())

# Create new columns for each key
for key in keys:
    proxy[key] = proxy['hyperparameters'].apply(lambda x: x.get(key, None))

# Drop the original "Hyperparameters" column
proxy_dataset = proxy.drop(columns=['hyperparameters'])
proxy_dataset['Label'] = proxy_dataset['Label'].astype(int)

In [18]:
param_grid = transform_grid(original_model.param_grid)
param_space, name = dimensions_aslists(param_grid)
space = Space(param_space)

plot_dims = []
for row in range(space.n_dims):
    if space.dimensions[row].is_constant:
        continue
    plot_dims.append((row, space.dimensions[row]))
iscat = [isinstance(dim[1], Categorical) for dim in plot_dims]
categorical = [name[i] for i,value in enumerate(iscat) if value == True]
proxy_dataset[categorical] = proxy_dataset[categorical].astype(str)

In [24]:
cat_transf = ColumnTransformer(transformers=[("cat", OneHotEncoder(), categorical)], remainder="passthrough")

proxy_model = Pipeline([
    ("one-hot", cat_transf),
    ("svm", SVC(kernel='linear', C=2.0 ,probability=True))
])

proxy_model = proxy_model.fit(proxy_dataset.drop(columns='Label'), proxy_dataset['Label'])

In [25]:
models = json.load(open("metadata/models.json"))
import joblib
# with open(models['Ideko_model']['cfs_surrogate_model'], 'wb') as f:
#     joblib.dump(proxy_model, models['Ideko_model']['cfs_surrogate_model'])  
# proxy_dataset.to_csv(models['Ideko_model']['cfs_surrogate_dataset'])
proxy_dataset = pd.read_csv(models['Ideko_model']['cfs_surrogate_dataset'],index_col=0)

In [18]:
query = pd.DataFrame(data = {'model__units': [params['model__units']], 'model__activation_function': params['model__activation_function'], 'batch_size':params['batch_size'],'epochs':params['epochs']})


In [19]:
query

,model__units,model__activation_function,batch_size,epochs
0,"[100, 100, 100]",tanh,16,50


In [49]:
e1.visualize_as_dataframe()

Query instance (original outcome : 0)


,batch_size,epochs,model__activation_function,model__units,BinaryLabel
0,16,50,tanh,"[100, 100, 100]",0



Diverse Counterfactual set (new outcome: 2)


,batch_size,epochs,model__activation_function,model__units,BinaryLabel
0,16,17,tanh,"[100, 100, 100]",2
1,16,16,tanh,"[25, 25, 25]",2
2,16,17,relu,"[100, 100, 100]",2
3,16,18,tanh,"[100, 100, 100]",2
4,16,15,tanh,"[100, 100, 100]",2
